<a href="https://colab.research.google.com/github/jsansao/transformers_pt/blob/main/BR_BERTo_Fine_tuning_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inferência usando transformers pré-treinados do HuggingFace 

In [1]:
#@title Passo 1:Instalando Hugging Face Transformers
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install transformers datasets
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.9.1
# tokenizers version at notebook update --- 0.7.0

Found existing installation: tensorflow 2.8.2+zzzcolab20220929150707
Uninstalling tensorflow-2.8.2+zzzcolab20220929150707:
  Successfully uninstalled tensorflow-2.8.2+zzzcolab20220929150707
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 4.9 MB/s 
     |████████████████████████████████| 432 kB 71.3 MB/s 
     |████████████████████████████████| 6.6 MB 47.9 MB/s 
     |████████████████████████████████| 163 kB 73.9 MB/s 
     |████████████████████████████████| 212 kB 73.3 MB/s 
     |████████████████████████████████| 115 kB 78.5 MB/s 
     |████████████████████████████████| 127 kB 73.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the foll

In [2]:
#@title Step 1: Loading the Dataset
#1.Load kant.txt using the Colab file manager
#2.Downloading the file from GitHub
!curl -L https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/dump_Machado_Nobreak.txt --output "dump.txt"
#!curl -L https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/Lyrics_ChicoBuarque.txt --output "kant.txt"
!awk NF < dump.txt > kant.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1739k  100 1739k    0     0  5721k      0 --:--:-- --:--:-- --:--:-- 5721k


In [3]:
#@title Passo 2:Baixando e salvando BR_BERTo
#https://huggingface.co/rdenadai/BR_BERTo

from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("rdenadai/BR_BERTo")

model = AutoModelForMaskedLM.from_pretrained("rdenadai/BR_BERTo")

Downloading:   0%|          | 0.00/516 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/695M [00:00<?, ?B/s]

In [4]:
!shuf -n1000 kant.txt > kant_teste.txt

In [5]:
from datasets import load_dataset
datasets = load_dataset("text", data_files={"train": "kant.txt", "validation": "kant_teste.txt"})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-6e3243dad78ca6fc/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [10]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

exemplo de entrada

In [14]:
tokenized_datasets["train"][4]

{'input_ids': [0, 37, 72, 23624, 478, 268, 28346, 2],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [17]:
#block_size = tokenizer.model_max_length
block_size = 32


In [18]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [19]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [20]:
#@title Step 11: Defining a Data Collator
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [21]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {metric.compute(predictions=predictions, references=labels)}

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
#@title Step 12: Initializing the Trainer
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./KantaiBERT",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],    
)

In [23]:
#@title Step 13: Pre-training the Model
%%time
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14920
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 9330
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,4.811700
1000,4.175400
1500,3.855600
2000,3.680300
2500,3.571700
3000,3.459500
3500,3.387200
4000,3.325500
4500,3.316400
5000,3.192900




Training completed. Do not forget to share your model on huggingface.co/models =)




CPU times: user 30min 49s, sys: 5min 30s, total: 36min 20s
Wall time: 36min 21s


TrainOutput(global_step=9330, training_loss=3.3883134422833767, metrics={'train_runtime': 2181.5443, 'train_samples_per_second': 68.392, 'train_steps_per_second': 4.277, 'total_flos': 1645639428096000.0, 'train_loss': 3.3883134422833767, 'epoch': 10.0})

In [24]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 1822
  Batch size = 8


Perplexity: 16.91


In [26]:
eval_results

{'eval_loss': 2.8281807899475098,
 'eval_runtime': 7.3847,
 'eval_samples_per_second': 246.726,
 'eval_steps_per_second': 30.875,
 'epoch': 10.0}

In [27]:
#@title Step 14: Saving the Final Model(+tokenizer + config) to disk
trainer.save_model("./KantaiBERT")

Saving model checkpoint to ./KantaiBERT
Configuration saved in ./KantaiBERT/config.json
Model weights saved in ./KantaiBERT/pytorch_model.bin


In [28]:
#@title Step 3: Configurando o pipeline fill-mask
#@title Step 15: Language Modeling with the FillMaskPipeline
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./KantaiBERT",
    tokenizer=tokenizer
)

loading configuration file ./KantaiBERT/config.json
Model config RobertaConfig {
  "_name_or_path": "./KantaiBERT",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 8,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.22.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 150000
}

loading configuration file ./KantaiBERT/config.json
Model config RobertaConfig {
  "_name_or_path": "./KantaiBERT",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_

In [30]:
fill_mask("O rapaz olhou para o <mask> ")

[{'score': 0.14221733808517456,
  'token': 3231,
  'token_str': ' quarto',
  'sequence': 'O rapaz olhou para o quarto '},
 {'score': 0.04389582946896553,
  'token': 1848,
  'token_str': ' médico',
  'sequence': 'O rapaz olhou para o médico '},
 {'score': 0.042837128043174744,
  'token': 89154,
  'token_str': ' tílburi',
  'sequence': 'O rapaz olhou para o tílburi '},
 {'score': 0.03788216412067413,
  'token': 20575,
  'token_str': ' agregado',
  'sequence': 'O rapaz olhou para o agregado '},
 {'score': 0.02788996882736683,
  'token': 3159,
  'token_str': ' amigo',
  'sequence': 'O rapaz olhou para o amigo '}]

In [32]:
fill_mask("Comprou uma <mask> na loja. ")

[{'score': 0.2077600061893463,
  'token': 683,
  'token_str': ' vez',
  'sequence': 'Comprou uma vez na loja. '},
 {'score': 0.03148023039102554,
  'token': 2620,
  'token_str': ' mão',
  'sequence': 'Comprou uma mão na loja. '},
 {'score': 0.029560307040810585,
  'token': 1663,
  'token_str': ' coisa',
  'sequence': 'Comprou uma coisa na loja. '},
 {'score': 0.028719110414385796,
  'token': 1739,
  'token_str': ' mulher',
  'sequence': 'Comprou uma mulher na loja. '},
 {'score': 0.027447490021586418,
  'token': 3031,
  'token_str': ' palavra',
  'sequence': 'Comprou uma palavra na loja. '}]

In [36]:
fill_mask("A mulher não é <mask>. ")

[{'score': 0.3035477101802826,
  'token': 1739,
  'token_str': ' mulher',
  'sequence': 'A mulher não é mulher. '},
 {'score': 0.07802890241146088,
  'token': 7567,
  'token_str': ' bonita',
  'sequence': 'A mulher não é bonita. '},
 {'score': 0.0674171969294548,
  'token': 1604,
  'token_str': ' homem',
  'sequence': 'A mulher não é homem. '},
 {'score': 0.0191931314766407,
  'token': 1662,
  'token_str': ' boa',
  'sequence': 'A mulher não é boa. '},
 {'score': 0.01656673289835453,
  'token': 13282,
  'token_str': ' casada',
  'sequence': 'A mulher não é casada. '}]

In [35]:
fill_mask("O homem não é <mask>. ")

[{'score': 0.14526425302028656,
  'token': 1604,
  'token_str': ' homem',
  'sequence': 'O homem não é homem. '},
 {'score': 0.04009541869163513,
  'token': 1739,
  'token_str': ' mulher',
  'sequence': 'O homem não é mulher. '},
 {'score': 0.03644940257072449,
  'token': 6219,
  'token_str': ' mau',
  'sequence': 'O homem não é mau. '},
 {'score': 0.027929676696658134,
  'token': 3640,
  'token_str': ' capaz',
  'sequence': 'O homem não é capaz. '},
 {'score': 0.01699405536055565,
  'token': 509,
  'token_str': ' seu',
  'sequence': 'O homem não é seu. '}]

In [40]:
fill_mask("Ele é um bom <mask>. ")

[{'score': 0.19883954524993896,
  'token': 3159,
  'token_str': ' amigo',
  'sequence': 'Ele é um bom amigo. '},
 {'score': 0.0964110940694809,
  'token': 1604,
  'token_str': ' homem',
  'sequence': 'Ele é um bom homem. '},
 {'score': 0.06384842097759247,
  'token': 5368,
  'token_str': ' advogado',
  'sequence': 'Ele é um bom advogado. '},
 {'score': 0.04947695508599281,
  'token': 2823,
  'token_str': ' ministro',
  'sequence': 'Ele é um bom ministro. '},
 {'score': 0.031973980367183685,
  'token': 6077,
  'token_str': ' padre',
  'sequence': 'Ele é um bom padre. '}]

In [41]:
fill_mask("Ela é uma boa <mask>. ")

[{'score': 0.19414366781711578,
  'token': 1507,
  'token_str': ' pessoa',
  'sequence': 'Ela é uma boa pessoa. '},
 {'score': 0.06419003754854202,
  'token': 1739,
  'token_str': ' mulher',
  'sequence': 'Ela é uma boa mulher. '},
 {'score': 0.04996379092335701,
  'token': 4418,
  'token_str': ' senhora',
  'sequence': 'Ela é uma boa senhora. '},
 {'score': 0.04990892857313156,
  'token': 5701,
  'token_str': ' amiga',
  'sequence': 'Ela é uma boa amiga. '},
 {'score': 0.033964380621910095,
  'token': 13121,
  'token_str': ' dama',
  'sequence': 'Ela é uma boa dama. '}]